In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('../data/pre_processed/data_covid_cases.parquet')

# Forecasting format

To main changes needed to be made when transitioning from the dataframe used when clustering to dataframe used for forecasting. The data was changed from accumulated infected to cases per day and the format of the dataframe was changed from 201 rows × 81 columns too 16281 rows × 3 columns.

Dataframe used as starting point:
<br>
Dataframe is pre-processed in pre_processing -> preporcessing notebook and saved in Data -> pre_processed

In [3]:
df.head(3)

,Algeria,Argentina,Australia,Austria,Bahrain,Bangladesh,Belarus,Belgium,Bolivia,Bulgaria,...,Tunisia,Turkey,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Vietnam,Zambia,Zimbabwe
2020-03-01,3.0,0.0,26.0,10.0,41.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,21.0,42.0,69.0,0.0,16.0,0.0,0.0
2020-03-02,3.0,0.0,29.0,14.0,47.0,0.0,1.0,19.0,0.0,0.0,...,0.0,0.0,0.0,21.0,47.0,89.0,0.0,16.0,0.0,0.0
2020-03-03,3.0,0.0,33.0,18.0,47.0,0.0,1.0,38.0,0.0,0.0,...,1.0,0.0,0.0,21.0,69.0,103.0,0.0,16.0,0.0,0.0


From accumulated cases too cases per day:

In [4]:
all_countries = list(df.columns)

In [5]:
df_mod = df.copy()

In [6]:
for country in all_countries:
    for i in range(1,201):
        df_mod[country][i] = df[country][i] - df[country][i-1]

In [7]:
df_mod.head(3)

,Algeria,Argentina,Australia,Austria,Bahrain,Bangladesh,Belarus,Belgium,Bolivia,Bulgaria,...,Tunisia,Turkey,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Vietnam,Zambia,Zimbabwe
2020-03-01,3.0,0.0,26.0,10.0,41.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,21.0,42.0,69.0,0.0,16.0,0.0,0.0
2020-03-02,0.0,0.0,3.0,4.0,6.0,0.0,0.0,18.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,20.0,0.0,0.0,0.0,0.0
2020-03-03,0.0,0.0,4.0,4.0,0.0,0.0,0.0,19.0,0.0,0.0,...,1.0,0.0,0.0,0.0,22.0,14.0,0.0,0.0,0.0,0.0


Creating a new dataframe with coulmns 'Cases' and 'Country':

In [8]:
new_df = pd.DataFrame(columns=['Cases','Country'])

In [9]:
new_df

,Cases,Country


Changing format of dataframe from having country as column name and cases as cell value to having Country as column name and cases as separate coulmn. This is to fit with the format used when forecasting

In [10]:
for country in all_countries:
    temp_segment = pd.DataFrame(df_mod[country])
    temp_segment.rename(columns={country:"Cases"}, inplace=True)
    temp_segment['Country']=country
    temp_segment['Date'] = temp_segment.index
    new_df = pd.concat([new_df, temp_segment], ignore_index = True)

In [11]:
new_df

,Cases,Country,Date
0,3.0,Algeria,2020-03-01
1,0.0,Algeria,2020-03-02
2,0.0,Algeria,2020-03-03
3,2.0,Algeria,2020-03-04
4,7.0,Algeria,2020-03-05
...,...,...,...
16678,29.0,Zimbabwe,2020-09-13
16679,18.0,Zimbabwe,2020-09-14
16680,5.0,Zimbabwe,2020-09-15
16681,45.0,Zimbabwe,2020-09-16


Saving dataframe to parquet

In [12]:
#new_df.to_parquet('../data/forecasting_format/cases_per_day_forecasting_format.parquet')